In [ ]:
# Getting turn of year deep in the money call greeks and prices.
# Goal is simple: get data for selected options on 12/31/2020 and then on current date.
# If easy, get daily prices for selected options.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn

In [ ]:
# Alt libraries:

import yfinance
import requests
import json

In [ ]:
import env

In [ ]:
import cboe_headers
# cboe_headers = {'Authorization': 'Bearer ############'} #This token expires after 30 minutes.

cboe_headers_code = cboe_headers.cboe_header_bearer

#### TDOC

In [ ]:
# Start date: 2021-01-04
# End date: 2021-02-25

start_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=AAPL&option_type=C&date=2021-02-01&min_expiry=2021-03-19&max_expiry=2021-03-19&min_strike=130&max_strike=130&symbol=AAPL'

end_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=AAPL&option_type=C&date=2021-02-24&min_expiry=2021-03-19&max_expiry=2021-03-19&min_strike=130&max_strike=130&symbol=AAPL'

headers = cboe_headers_code

In [ ]:
base_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'

In [ ]:
tdoc_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=TDOC&option_type=C&date=2021-01-04&min_expiry=2022-01-21&max_expiry=2022-01-21&min_strike=130&max_strike=150&symbol=TDOC'

In [ ]:
# TDOC start data:

root = 'TDOC'
option_type = 'C'
date = '2021-01-04'
min_expiry = '2022-01-21'
max_expiry = '2022-01-21'
min_strike = '145'
max_strike = '145'
symbol = root

tdoc_start_url = f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'

# tdoc_start_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=TDOC&option_type=C&date=2021-01-04&min_expiry=2022-01-21&max_expiry=2022-01-21&min_strike=130&max_strike=150&symbol=TDOC'

In [ ]:
tdoc_sdata = requests.get(url = tdoc_start_url, headers = headers)
tdoc_start_data = json.loads(tdoc_sdata.text)

In [ ]:
tdoc_start_data

In [ ]:
# TDOC end data:

root = 'TDOC'
option_type = 'C'
date = '2021-02-25'
min_expiry = '2022-01-21'
max_expiry = '2022-01-21'
min_strike = '145'
max_strike = '145'
symbol = root

tdoc_end_url = f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'

tdoc_edata = requests.get(url = tdoc_end_url, headers = headers)
tdoc_end_data = json.loads(tdoc_edata.text)
tdoc_end_data

In [ ]:
# The Dateframe:

tdoc_df_test = pd.DataFrame(tdoc_end_data['options'][0], index = [0])
columns_to_keep = ['root', 'expiry', 'strike', 'option_type', 'timestamp', 'option_mid', 'delta']

tdoc_df_test = tdoc_df_test[[c for c in tdoc_df_test.columns if c in columns_to_keep]]
tdoc_df_test

In [ ]:
tdoc_df_test.T

In [ ]:
# Creating a function for the start date call:

# root = 'TDOC'
# option_type = 'C'
# date = '2021-01-04'
# min_expiry = '2022-01-21'
# max_expiry = '2022-01-21'
# min_strike = '145'
# max_strike = '145'
# symbol = root


def start_data(root, option_type, date, min_expiry, max_expiry, min_strike, max_strike, symbol, header_code):
    
    url = f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'
    
    cboe_headers = {'Authorization': f'Bearer {header_code}'}
    
    
    # Getting the data from the API:
    stock_get = requests.get(url = url, headers = headers)
    
    stock_data = json.loads(stock_get.text)
    
    # Creating the Dataframe and cleaning up columns:
    stock_df = pd.DataFrame(stock_data['options'][0], index = [0])
    columns_to_keep = ['root', 'expiry', 'strike', 'option_type', 'timestamp', 'option_mid', 'option_bid', 'option_ask', 'delta', 'gamme', 'vega', 'theta', 'rho']
    
    # Using list comprehension to filter for the columns I want to keep
    stock_df = stock_df[[c for c in stock_df.columns if c in columns_to_keep]]
    
    # Adding the as of date:
    stock_df['as_of_date'] = date
    
    # Returning the df:
    
    return stock_df
    

In [ ]:
testing_df = start_data('TDOC', "C", '2021-01-04', '2022-01-21', '2022-01-21', '130', '145', 'TDOC', '89c716765c54c540afd19ce46eec6dfa')

In [ ]:
testing_df

In [ ]:
date_test = '2021-01-04'

In [ ]:
testing_df['date_as_of'] = date_test

In [ ]:
testing_df

In [ ]:
testing_df.info()